In [80]:
import json
import pandas as pd

In [81]:
# JSON-Datei laden
with open('generated_responses_few_errors.json', 'r') as f:
    data = json.load(f)

# Daten in ein DataFrame umwandeln
rows = []
for entry in data:
    question_id = entry["question_id"]
    question = entry["question"]
    question_type = entry["type"]
    for answer in entry["answers"]:
        rows.append({
            "question_id": question_id,
            "question": question,
            "type": question_type,
            "answer_text": answer["text"],
            "answer_label": answer["label"],
            "timestamp": answer["timestamp"]
        })

# DataFrame erstellen
df = pd.DataFrame(rows)

# DataFrame anzeigen
#print(df.head())

In [82]:
df_single_select = df[df["type"] == "SINGLE_SELECT"]

In [83]:
df_single_select

,question_id,question,type,answer_text,answer_label,timestamp
0,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,Yes.,Yes,2025-01-21T10:36:32.886790
1,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,"No, I do not consent to the processing of my d...",No,2025-01-21T10:36:38.603820
2,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,"Yes, you may process my data.",Yes,2025-01-21T10:36:44.270522
3,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,"No, I do not consent.",No,2025-01-21T10:36:49.886813
4,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,I don't know,No Preference,2025-01-21T10:36:49.886866
...,...,...,...,...,...,...
695,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,I don't know,No Preference,2025-01-21T11:29:45.005852
696,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,"Yes, you may process my data as described in y...",Yes,2025-01-21T11:29:50.695825
697,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,,No Preference,2025-01-21T11:29:50.695881
698,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,May we process your data?,SINGLE_SELECT,"Yes, you may process my data.",Yes,2025-01-21T11:29:56.288688


In [84]:
import google.generativeai as genai

In [85]:
genai.configure(api_key="")
#model=genai.GenerativeModel("gemini-1.5-flash")

In [86]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = genai.generate_text(prompt="Hi")

NotFound: 404 Requested entity was not found.

In [60]:
import datetime  # Für den echten Timestamp

In [61]:
""" def generate_answer(question, label):
    prompt = f"Generate diverse responses for the following question with the answer label '{label}': {question}"

    # API-Aufruf an das Gemini Modell
    response = genai.generate_text(
        #model="gemini-1.5-flash",  # Wähle das gewünschte Gemini Modell
        prompt=prompt,
        temperature=0.7 
    )
    
    generated_answer = response.text
    return generated_answer
 """

' def generate_answer(question, label):\n    prompt = f"Generate diverse responses for the following question with the answer label \'{label}\': {question}"\n\n    # API-Aufruf an das Gemini Modell\n    response = genai.generate_text(\n        #model="gemini-1.5-flash",  # Wähle das gewünschte Gemini Modell\n        prompt=prompt,\n        temperature=0.7 \n    )\n    \n    generated_answer = response.text\n    return generated_answer\n '

In [62]:
def rephrase_question(question, label):
    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        prompt = f"Generate diverse responses for the following question with the answer label '{label}': {question}"
        #prompt = f"Rephrase this survey question professionally: {text}"
        response = genai.generate_text(
        #model="gemini-1.5-flash",  # Wähle das gewünschte Gemini Modell
            prompt=prompt,
            temperature=0.7 
        )
        return response.text.strip()
    except Exception as e:
        print(f"Error with Gemini API: {e}")
        return text  # Fallback to original if API fails

In [63]:
""" # Funktion zur Generierung diverser Antworten mit Gemini Flash 1.5 (oder OpenAI)
def generate_answer(question, label):
    prompt = f"Generate diverse responses for the following question with the answer label '{label}': {question}"

    # API-Aufruf an das Modell (hier beispielhaft mit OpenAI, ersetze es durch Gemini Flash 1.5, falls erforderlich)
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",  # Beispielmodell, ersetze es durch Gemini Flash 1.5, falls erforderlich
        prompt=prompt,
        max_tokens=60,
        temperature=0.7  # Temperatur steuert die Kreativität der Antwort
    )
    
    generated_answer = response.choices[0].text.strip()
    return generated_answer """

# Funktion zur Generierung diverser Antworten für das gesamte DataFrame
def generate_diverse_answers(df):
    generated_answers = []
    
    for _, row in df.iterrows():
        question = row['question']
        label = row['answer_label']
        
        # Generiere eine Antwort mit dem Modell
        generated_answer = generate_answer(question, label)
        print(f"Question: {question}\nAnswer: {generate_answer}\nLabel: {label}\n")
        
        # Aktuellen Timestamp generieren
        timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
        
        # Neue Zeile zu den generierten Antworten hinzufügen
        generated_answers.append({
            'question_id': row['question_id'],
            'question': question,
            'type': row['type'],
            'answer_text': generated_answer,
            'answer_label': label,
            'timestamp': timestamp  # Aktueller Timestamp
        })
    
    # Rückgabe des erweiterten DataFrames
    return pd.DataFrame(generated_answers)


In [64]:
# Generiere diverse Antworten und erweitere den DataFrame
df_extended = generate_diverse_answers(df_single_select)

# Ausgabe des erweiterten DataFrames
print(df_extended)

NotFound: 404 Requested entity was not found.

In [ ]:
#pip install google-generativeai